## Data Cleaning Function
%load_ext lab_black

In [1]:
import pandas as pd
import random
import numpy as np

In [2]:
# load mortality the data

death03 = pd.read_csv(
    "./US_VitalStatistics/Underlying Cause of Death, 2003.txt",
    error_bad_lines=False,
    sep="\t",
)
death04 = pd.read_csv(
    "./US_VitalStatistics/Underlying Cause of Death, 2004.txt",
    error_bad_lines=False,
    sep="\t",
)
death05 = pd.read_csv(
    "./US_VitalStatistics/Underlying Cause of Death, 2005.txt",
    error_bad_lines=False,
    sep="\t",
)
death06 = pd.read_csv(
    "./US_VitalStatistics/Underlying Cause of Death, 2006.txt",
    error_bad_lines=False,
    sep="\t",
)
death07 = pd.read_csv(
    "./US_VitalStatistics/Underlying Cause of Death, 2007.txt",
    error_bad_lines=False,
    sep="\t",
)
death08 = pd.read_csv(
    "./US_VitalStatistics/Underlying Cause of Death, 2008.txt",
    error_bad_lines=False,
    sep="\t",
)
death09 = pd.read_csv(
    "./US_VitalStatistics/Underlying Cause of Death, 2009.txt",
    error_bad_lines=False,
    sep="\t",
)
death10 = pd.read_csv(
    "./US_VitalStatistics/Underlying Cause of Death, 2010.txt",
    error_bad_lines=False,
    sep="\t",
)
death11 = pd.read_csv(
    "./US_VitalStatistics/Underlying Cause of Death, 2011.txt",
    error_bad_lines=False,
    sep="\t",
)
death12 = pd.read_csv(
    "./US_VitalStatistics/Underlying Cause of Death, 2012.txt",
    error_bad_lines=False,
    sep="\t",
)
death13 = pd.read_csv(
    "./US_VitalStatistics/Underlying Cause of Death, 2013.txt",
    error_bad_lines=False,
    sep="\t",
)
death14 = pd.read_csv(
    "./US_VitalStatistics/Underlying Cause of Death, 2014.txt",
    error_bad_lines=False,
    sep="\t",
)
death15 = pd.read_csv(
    "./US_VitalStatistics/Underlying Cause of Death, 2015.txt",
    error_bad_lines=False,
    sep="\t",
)

In [3]:
df = pd.concat(
    [death03, death04, death05, death06, death07, death08, death09, death10, death11, death12, death13, death14, death15],
    ignore_index=True,
)

In [4]:
def clean_mortality(df, State_Abbr):
    df[["County1", "State"]] = df.County.str.split(", ", expand=True)
    df = df[df["State"] == State_Abbr]
    df = df.drop(["Notes", "County", "County Code", "Year Code"], axis=1)
    df = df.rename(columns={"County1": "County"})
    df = df[(df["Drug/Alcohol Induced Cause Code"] == "D1") | (df["Drug/Alcohol Induced Cause Code"] == "D2") | (df["Drug/Alcohol Induced Cause Code"] == "D4") | (df["Drug/Alcohol Induced Cause Code"] == "D9")]
    df["Deaths"] = df["Deaths"].astype(float)
    df = df.groupby(["Year", "State", "County"], as_index=False).sum()
    df["Year"] = df["Year"].astype(int)
    return df

In [5]:
fl_mortality = clean_mortality(df, "FL")

In [6]:
def clean_pop(df1,df2,State_ABBR):
    df1 = df1.drop(["Unnamed: 1", 2000, 2001, 2002, "Unnamed: 12", "Unnamed: 13"], axis=1)
    df1 = df1.rename(columns={"Unnamed: 0": "County"})
    df2 = df2.drop(["Census", "Estimates Base", 2016, 2017, 2018, 2019], axis=1)
    df2 = df2.rename(columns={"Unnamed: 0": "County"})
    df1.drop(df1.head(1).index, inplace=True)
    df1.drop(df1.tail(8).index, inplace=True)
    df2.drop(df2.head(1).index, inplace=True)
    df2.drop(df2.tail(5).index, inplace=True)
    df1["County"] = df1["County"].str[1:]
    df2["County"] = df2["County"].str[1:]
    df1 = df1.melt(id_vars=["County"])
    df1 = df1.rename(columns={"variable": "Year", "value": "Population"})
    df1 = df1.groupby(["Year", "County"], as_index=False).sum()
    df2 = df2.melt(id_vars=["County"])
    df2 = df2.rename(columns={"variable": "Year", "value": "Population"})
    df2 = df2.groupby(["Year", "County"], as_index=False).sum()
    df2[["County1", "State"]] = df2.County.str.split(", ", expand=True)
    df2 = df2.drop(["County", "State"], axis=1)
    df2 = df2.rename(columns={"County1": "County"})
    df_concat = pd.concat([df1,df2],ignore_index=True)
    df_concat["State"] = State_ABBR
    return df_concat

In [7]:
fl_pop_pre_policy = pd.read_excel("./Population/fl_pop_2000-2010.xls", header=[3])
fl_pop_post_policy = pd.read_excel("./Population/fl_pop_2010-2019.xlsx", header=[3])

In [8]:
fl_pop = clean_pop(fl_pop_pre_policy,fl_pop_post_policy, "FL")

In [9]:
# Function that merges population data with mortality data
def merge_mortalitypop(df_mortality, df_pop):
    df_pop["County"] = df_pop["County"].astype("string")
    df_pop["State"] = df_pop["State"].astype("string")
    df_mortality["County"] = df_mortality["County"].astype("string")
    df_mortality["State"] = df_mortality["State"].astype("string")
    merged = df_mortality.merge(df_pop,left_on=["Year", "County"],
    right_on=["Year", "County"],
    how="right",
)
    merged = merged.drop(["State_x"], axis=1)
    merged = merged.rename(columns={"State_y": "State"})
    merged["Deaths"] = merged["Deaths"].apply(
    lambda l: l if not np.isnan(l) else np.random.randint(0, 9)
)
    merged["Mortality Rate"] = merged["Deaths"] / merged["Population"]
    return merged


# Function that groups dataframe by year, and calculate the total death and total population
def calc_avg_mortality(df1, df2, df3):
    df1 = df1.groupby('Year', as_index=False).sum()[['Year', 'Deaths', 'Population']]
    df2 = df2.groupby('Year', as_index=False).sum()[['Year', 'Deaths', 'Population']]
    df3 = df3.groupby('Year', as_index=False).sum()[['Year', 'Deaths', 'Population']]

    df = pd.concat([df1, df2, df3], ignore_index=True)
    df = df.groupby('Year', as_index=False).sum()
    df['Avg Mortality Rate'] = df['Deaths'] / df['Population']
    return df

In [10]:
test = merge_mortalitypop(fl_mortality, fl_pop)
test

,Year,County,Deaths,Population,State,Mortality Rate
0,2003,Alachua County,11.0,227022.0,FL,0.000048
1,2003,Baker County,7.0,23555.0,FL,0.000297
2,2003,Bay County,21.0,155044.0,FL,0.000135
3,2003,Bradford County,4.0,27035.0,FL,0.000148
4,2003,Brevard County,97.0,504847.0,FL,0.000192
...,...,...,...,...,...,...
866,2015,Union County,5.0,15265.0,FL,0.000328
867,2015,Volusia County,98.0,516730.0,FL,0.000190
868,2015,Wakulla County,4.0,31523.0,FL,0.000127
869,2015,Walton County,1.0,63066.0,FL,0.000016


In [11]:
calc_avg_mortality(test,test,test)

,Year,Deaths,Population,Avg Mortality Rate
0,2003,5847.0,51012255.0,0.000115
1,2004,6543.0,52245954.0,0.000125
2,2005,7260.0,53526114.0,0.000136
3,2006,7932.0,54500970.0,0.000146
4,2007,8094.0,55103526.0,0.000147
5,2008,8574.0,55581915.0,0.000154
6,2009,8841.0,55957932.0,0.000158
7,2010,8700.0,56536611.0,0.000154
8,2011,8412.0,57159711.0,0.000147
9,2012,7398.0,57893466.0,0.000128
